In [1]:
import dash
# import dash_core_components as dcc
from dash import dcc
# import dash_html_components as html
from dash import html
from dash import dash_table
import dash_bootstrap_components as dbc

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import date

In [3]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MATERIA])

IMPORT INCIDENT TICKETS

In [4]:
# Create initial Dataframe
df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/INC_YTD_nov2022.csv')
# df_incident.info()

In [5]:
# Replace NaN values with '01-01-2200' 
df_incident['resolved_at'].fillna('01/01/2200', inplace=True)
df_incident['closed_at'].fillna('01/01/2200', inplace=True)
# df_incident.info()

In [6]:
# Save new Dataframe 
df_incident.to_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', index=False)

In [7]:
# Recreate Dataframe and conver Date columns to Datetime format
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at', 'resolved_at'],
                            date_parser = d_parser , encoding = 'unicode_escape')
                            
# df_incident.head()

C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_16224\1158756277.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_16224\1158756277.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_16224\1158756277.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')


IMPORT TASK TICKETS

In [8]:
# Create initial Dataframe
df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/TASK_YTD_nov2022.csv')
# df_task.info()

In [9]:
# Fill any null values
df_task['closed_at'].fillna('01/01/2200', inplace=True)
# df_task.info()

In [10]:
# Save new Dataframe 
df_task.to_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', index=False)

In [11]:
# Create SCTASK DataFrame
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at'], date_parser = d_parser, encoding='unicode_escape')

# df_task.head()

C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_16224\2542818920.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_16224\2542818920.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')


GLOBAL VARIABLES

In [12]:
month_date_today = date.today()

# Get first day of current month
split_month_date_today = str(month_date_today).split('-')
split_month_date_today[2] = '01'
month_first_day = '-'.join([str(elem) for elem in split_month_date_today])

# Get first day of last month
get_last_month = str(int(split_month_date_today[1]) - 1 )
split_month_date_today[1] = get_last_month
last_month = '-'.join([str(elem) for elem in split_month_date_today])

# Get January 01 of current year
split_month_date_today[1] = '01'
year_first_day = '-'.join([str(elem) for elem in split_month_date_today])

research_assessment_types = df_incident["u_research_assessment"].unique().tolist()

# print(f"month_date_today: {month_date_today}")
# print(f"split_month_date_today: {split_month_date_today}")
# print(f"month_first_day: {month_first_day}")
# print(f"get_last_month: {get_last_month}")
# print(f"last_month: {last_month}")
print(f"year_first_day: {year_first_day}")



year_first_day: 2022-01-01


CARD COUNTS - INC Opened and Resolved

In [13]:
def ticket_count_opened(timeline, df):    
    count = 0

    if timeline == 'current-month':
        date_filter = (df['opened_at'] >= month_first_day)
        count = len(date_filter.loc[date_filter])
        return count
    elif timeline == 'last-month':
        date_filter = (df['opened_at'] < month_first_day) & (df['opened_at'] >= last_month)
        count = len(date_filter.loc[date_filter])
        return count
    elif timeline == 'current-year':
        count = df.loc[df['opened_at'] >= year_first_day].shape[0]
        return count
    elif timeline == 'ytd':
        count = df.shape[0]
        return count

def ticket_count_resolved(timeline, state, df):    
    count = 0

    if timeline == 'current-month':
        date_filter = (df[state] >= month_first_day) & (df[state] < '2200-01-01')
        count = len(date_filter.loc[date_filter])
        return count
    elif timeline == 'last-month':
        date_filter = (df[state] < month_first_day) & (df[state] >= last_month)
        count = len(date_filter.loc[date_filter])
        return count
    elif timeline == 'current-year':
        date_filter = (df[state] >= year_first_day) & (df[state] < '2200-1-1')
        count = df.loc[date_filter].shape[0]
        return count
    elif timeline == 'ytd':
        date_filter = (df[state] < '2200-1-1')
        count = df.loc[date_filter].shape[0]
        return count

In [14]:
inc_count_opened_cm = ticket_count_opened('current-month', df_incident)
inc_count_opened_lm = ticket_count_opened('last-month', df_incident)
inc_count_opened_cy = ticket_count_opened('current-year', df_incident)
inc_count_opened_ytd = ticket_count_opened('ytd', df_incident)

task_count_opened_cm = ticket_count_opened('current-month', df_task)
task_count_opened_lm = ticket_count_opened('last-month', df_task)
task_count_opened_cy = ticket_count_opened('current-year', df_task)
task_count_opened_ytd = ticket_count_opened('ytd', df_task)

# print(f"inc_count_opened_cm: {inc_count_opened_cm}")
# print(f"inc_count_opened_lm: {inc_count_opened_lm}")
# print(f"inc_count_opened_cy: {inc_count_opened_cy}")
# print(f"inc_count_opened_ytd: {inc_count_opened_ytd}")
# print("-----------------------")
# print(f"task_count_opened_cm: {task_count_opened_cm}")
# print(f"task_count_opened_lm: {task_count_opened_lm}")
# print(f"task_count_opened_cy: {task_count_opened_cy}")
# print(f"task_count_opened_ytd: {task_count_opened_ytd}")

In [15]:
inc_count_resolved_cm = ticket_count_resolved('current-month', 'resolved_at', df_incident)
inc_count_resolved_lm = ticket_count_resolved('last-month', 'resolved_at', df_incident)
inc_count_resolved_cy = ticket_count_resolved('current-year', 'resolved_at', df_incident) 
inc_count_resolved_ytd = ticket_count_resolved('ytd', 'resolved_at', df_incident) 


task_count_resolved_cm = ticket_count_resolved('current-month', 'closed_at', df_task)
task_count_resolved_lm = ticket_count_resolved('last-month', 'closed_at', df_task)
task_count_resolved_cy = ticket_count_resolved('current-year', 'closed_at', df_task)
task_count_resolved_ytd = ticket_count_resolved('ytd', 'closed_at', df_task)

# print(f"inc_count_resolved_cm: {inc_count_resolved_cm}")
# print(f"inc_count_resolved_lm: {inc_count_resolved_lm}")
# print(f"inc_count_resolved_cy: {inc_count_resolved_cy}")
# print(f"inc_count_resolved_ytd: {inc_count_resolved_ytd}")
# print("-----------------------")
# print(f"task_count_resolved_cm: {task_count_resolved_cm}")
# print(f"task_count_resolved_lm: {task_count_resolved_lm}")
# print(f"task_count_resolved_cy: {task_count_resolved_cy}")
# print(f"task_count_resolved_ytd: {task_count_resolved_ytd}")

Volume of Incidents by Issue Type

In [16]:
# NOTE: Blank Research assessment needs to be addressed

def inc_issue_type(timeline):
    df_research_assessment = ""
    research_assessment_count = []

    if timeline == 'current-month':
        date_filter = (df_incident['opened_at'] >= month_first_day)
        df_filtered = df_incident.loc[date_filter]

        # Get Count for each 'Research Assessment'
        
        for issue_type in research_assessment_types:
            count = df_filtered.loc[df_filtered["u_research_assessment"] == issue_type].shape[0]
            research_assessment_count.append(count)

        # Create DataFrame for Research Assessment Count
        df_research_assessment = pd.DataFrame({
            "Research Assessment": research_assessment_types,
            "": research_assessment_count
        })

        df_research_assessment = df_research_assessment.sort_values('', ascending=True)
        return df_research_assessment

    elif timeline == 'last-month':
        date_filter = (df_incident['opened_at'] < month_first_day) & (df_incident['opened_at'] >= last_month)
        df_filtered = df_incident.loc[date_filter]

        # Get Count for each 'Research Assessment'
        
        for issue_type in research_assessment_types:
            count = df_filtered.loc[df_filtered["u_research_assessment"] == issue_type].shape[0]
            research_assessment_count.append(count)
            # print(f"issue_type: {issue_type} - {count}")

        # Create DataFrame for Research Assessment Count
        df_research_assessment = pd.DataFrame({
            "Research Assessment": research_assessment_types,
            "": research_assessment_count
        })

        df_research_assessment = df_research_assessment.sort_values('', ascending=True)
        return df_research_assessment

    elif timeline == 'current-year':
        first_month = month_first_day.replace(str(f"-{split_month_date_today[1]}"), str(f"-01"))
        date_filter = (df_incident['opened_at'] < '2200-01-01') & (df_incident['opened_at'] >= first_month)
        df_filtered = df_incident.loc[date_filter]

        # Get Count for each 'Research Assessment'
        
        for issue_type in research_assessment_types:
            count = df_filtered.loc[df_filtered["u_research_assessment"] == issue_type].shape[0]
            research_assessment_count.append(count)

        # Create DataFrame for Research Assessment Count
        df_research_assessment = pd.DataFrame({
            "Research Assessment": research_assessment_types,
            "": research_assessment_count
        })

        df_research_assessment = df_research_assessment.sort_values('', ascending=True)
        return df_research_assessment
        
    elif timeline == 'ytd':
        for issue_type in research_assessment_types:
            count = df_incident.loc[df_incident["u_research_assessment"] == issue_type].shape[0]
            research_assessment_count.append(count)

        # Create DataFrame for Research Assessment Count
        df_research_assessment = pd.DataFrame({
            "Research Assessment": research_assessment_types,
            "": research_assessment_count
        })

        df_research_assessment = df_research_assessment.sort_values('', ascending=True)
        return df_research_assessment

df_assessment_cm = inc_issue_type('current-month')
df_assessment_lm = inc_issue_type('last-month')
df_assessment_cy = inc_issue_type('current-year')
df_assessment_ytd = inc_issue_type('ytd')


In [17]:
fig_df_assessment_cm = px.bar(df_assessment_cm, x="", y="Research Assessment", barmode="group", text_auto=True, orientation="h")
fig_df_assessment_lm = px.bar(df_assessment_lm, x="", y="Research Assessment", barmode="group", text_auto=True, orientation="h")
fig_df_assessment_cy = px.bar(df_assessment_cy, x="", y="Research Assessment", barmode="group", text_auto=True, orientation="h")
fig_df_assessment_ytd = px.bar(df_assessment_ytd, x="", y="Research Assessment", barmode="group", text_auto=True, orientation="h")

CARD

In [18]:
def card_count(title, title_size, data, data_size):
    card = dbc.Card(
            dbc.CardBody(
                [
                    html.H5(title, className="card-title", style={'font-size': title_size}),
                    html.P(data, style={'font-size': data_size, 'font-weight': 'bold', 'text-align':'center', 'margin-bottom':'0rem'}),
                    
                ]
            )
        )
    
    return card

# For Current Month
card_inc_opened_cm = card_count('Incidents Opened', '1.5rem', inc_count_opened_cm, '10rem')
card_inc_resolved_cm = card_count('Incidents Resolved', '1.5rem', inc_count_resolved_cm, '10rem')
card_task_opened_cm = card_count('Tasks Opened', '1.5rem', task_count_opened_cm, '10rem')
card_task_resolved_cm = card_count('Tasks Resolved', '1.5rem', task_count_resolved_cm, '10rem')

row1_cm = dbc.Row(
        [
            dbc.Col(card_inc_opened_cm, width = 3),
            dbc.Col(card_inc_resolved_cm, width = 3),
            dbc.Col(card_task_opened_cm, width = 3),
            dbc.Col(card_task_resolved_cm, width = 3),
        ]
    )

# For Last Month
card_inc_opened_lm = card_count('Incidents Opened', '1.5rem', inc_count_opened_lm, '10rem')
card_inc_resolved_lm = card_count('Incidents Resolved', '1.5rem', inc_count_resolved_lm, '10rem')
card_task_opened_lm = card_count('Tasks Opened', '1.5rem', task_count_opened_lm, '10rem')
card_task_resolved_lm = card_count('Tasks Resolved', '1.5rem', task_count_resolved_lm, '10rem')

row1_lm = dbc.Row(
        [
            dbc.Col(card_inc_opened_lm, width = 3),
            dbc.Col(card_inc_resolved_lm, width = 3),
            dbc.Col(card_task_opened_lm, width = 3),
            dbc.Col(card_task_resolved_lm, width = 3),
        ]
    )

# For Current Year
card_inc_opened_cy = card_count('Incidents Opened', '1.5rem', inc_count_opened_cy, '10rem')
card_inc_resolved_cy = card_count('Incidents Resolved', '1.5rem', inc_count_resolved_cy, '10rem')
card_task_opened_cy = card_count('Tasks Opened', '1.5rem', task_count_opened_cy, '10rem')
card_task_resolved_cy = card_count('Tasks Resolved', '1.5rem', task_count_resolved_cy, '10rem')

row1_cy = dbc.Row(
        [
            dbc.Col(card_inc_opened_cy, width = 3),
            dbc.Col(card_inc_resolved_cy, width = 3),
            dbc.Col(card_task_opened_cy, width = 3),
            dbc.Col(card_task_resolved_cy, width = 3),
        ]
    )

# For YTD
card_inc_opened_ytd = card_count('Incidents Opened', '1.5rem', inc_count_opened_ytd, '10rem')
card_inc_resolved_ytd = card_count('Incidents Resolved', '1.5rem', inc_count_resolved_ytd, '10rem')
card_task_opened_ytd = card_count('Tasks Opened', '1.5rem', task_count_opened_ytd, '10rem')
card_task_resolved_ytd = card_count('Tasks Resolved', '1.5rem', task_count_resolved_ytd, '10rem')

row1_ytd = dbc.Row(
        [
            dbc.Col(card_inc_opened_ytd, width = 3),
            dbc.Col(card_inc_resolved_ytd, width = 3),
            dbc.Col(card_task_opened_ytd, width = 3),
            dbc.Col(card_task_resolved_ytd, width = 3),
        ]
    )

Tickets Opened and Resolved

In [19]:
def year_month_data(selected_year, state, df):

    initial_year_month_list = df[state].dt.strftime('%Y-%m').unique().tolist()
    year_month_list = []

    for x in initial_year_month_list:
        if selected_year in x:
            year_month_list.append(x)

    # Get available month names for selected year
    get_month_names = df.loc[df[state] > selected_year][state].dt.strftime('%b').unique().tolist()

    return year_month_list, get_month_names
    
inc_ym_2022, inc_mn_2022 = year_month_data('2022', 'opened_at', df_incident)

# print(inc_ym_2022)
# print(inc_mn_2022)

In [20]:
def year_month_count_opened(year_month_list, state, df):
    year_month_count_opened = []

    for get_curr_year_month in year_month_list:

        get_curr_month = get_curr_year_month.split('-')[1]
        get_curr_year = get_curr_year_month.split('-')[0]
        get_next_month = int(get_curr_month) + 1
        get_next_year = int(get_curr_year) + 1

        if get_next_month > 12:
            
            get_next_month = 1;
            get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
            get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

            count = df.loc[(df[state] < f"{get_next_year_month}-01") & 
                                    (df[state] >= f"{get_curr_year_month}-01")]

            year_month_count_opened.append(count.shape[0])
        else:
            get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

            count = df.loc[(df[state] < f"{get_next_year_month}-01") & 
                                    (df[state] >= f"{get_curr_year_month}-01")]

            year_month_count_opened.append(count.shape[0])

    return year_month_count_opened

inc_ymc_opened_2022 = year_month_count_opened(inc_ym_2022, 'opened_at' ,df_incident)
inc_ymc_resolved_2022 = year_month_count_opened(inc_ym_2022, 'resolved_at' ,df_incident)

task_ymc_opened_2022 = year_month_count_opened(inc_ym_2022, 'opened_at' ,df_task)
task_ymc_resolved_2022 = year_month_count_opened(inc_ym_2022, 'closed_at' ,df_task)

# print(f"inc_ymc_opened_2022: {inc_ymc_opened_2022}")
# print(f"inc_ymc_resolved_2022: {inc_ymc_resolved_2022}")
# print(f"task_ymc_opened_2022: {task_ymc_opened_2022}")
# print(f"task_ymc_resolved_2022 : {task_ymc_resolved_2022}")


In [21]:
# Create DataFrame for 'opened_at' column based on 'year_month_list' and 'year_month_count'
df_inc_open_and_close = pd.DataFrame({
    "month_year": inc_ym_2022,
    "count_open": inc_ymc_opened_2022,
    "count_resolved": inc_ymc_resolved_2022
})

# df_inc_open_and_close

In [22]:
# Create figure for 'df_inc_open_and_close'
colors = px.colors.qualitative.Set3

fig2 = go.Figure(data=[
    go.Bar(name='Opened', x=inc_mn_2022, y=inc_ymc_opened_2022, text=inc_ymc_opened_2022, marker=dict(color=colors[4])),
    go.Bar(name='Resolved', x=inc_mn_2022, y=inc_ymc_resolved_2022, text=inc_ymc_resolved_2022, marker=dict(color=colors[3])),
])

# fig2.show()

INC & TASK OPENED

In [23]:
# # Create DataFrame
df_inc_and_task_open = pd.DataFrame({
    "month_year": inc_ym_2022,
    "count_open_inc": inc_ymc_opened_2022,
    "count_open_task": task_ymc_opened_2022,
})

# print(df_inc_and_task_open)

In [24]:
# # Create figure for 'df_inc_and_task_open'
colors = px.colors.qualitative.Set3

fig3 = go.Figure(data=[
    go.Bar(name='Incidents', x=inc_mn_2022, y=inc_ymc_opened_2022, text=inc_ymc_opened_2022, marker=dict(color=colors[4])),
    go.Bar(name='Tasks', x=inc_mn_2022, y=task_ymc_opened_2022, text=task_ymc_opened_2022, marker=dict(color=colors[3])),
])

# fig3

In [25]:
# Filler Pie Chart
df_pie = px.data.tips()
fig_pie = px.pie(df_pie, values='tip', names='day', color='day',
             color_discrete_map={'Thur':'lightcyan',
                                 'Fri':'cyan',
                                 'Sat':'royalblue',
                                 'Sun':'darkblue'})
# fig_pie.show()

INC BY T-SHIRT SIZE

In [26]:
def countsize_per_type(df):
    s = []
    m = []
    l = []

    research_assessment_types = df["u_research_assessment"].unique().tolist()
    for type in research_assessment_types:
            
            count_s = df.loc[(df['u_research_assessment'] == type) & (df['correlation_display'] == 'S')].shape[0]
            count_m = df.loc[(df['u_research_assessment'] == type) & (df['correlation_display'] == 'M')].shape[0]
            count_l = df.loc[(df['u_research_assessment'] == type) & (df['correlation_display'] == 'L')].shape[0]

            s.append(count_s)
            m.append(count_m)
            l.append(count_l)

    return s, m, l, research_assessment_types

def create_trbs_df(date_filter):
    df = df_incident.loc[date_filter]
    s, m, l, research_assessment_types = countsize_per_type(df)
    
    data = {
        'Research Assessment': research_assessment_types,
        'S': s,
        'M': m,
        'L': l
    }

    df2 = pd.DataFrame(data)
    df2['Count'] = df2.iloc[:, 1:4].sum(axis=1) # Add column for row Count
    df2.loc[len(df2.index)] = ['Total', df2['S'].sum(), df2['M'].sum(), df2['L'].sum(), df2['Count'].sum()] # Add row for column Total
    return df2

def df_ticket_resolved_by_size(timeline):    

    if timeline == 'current-month':

        date_filter = (df_incident['resolved_at'] >= month_first_day) & (df_incident['resolved_at'] < '2200-01-01')
        df = create_trbs_df(date_filter)
        return df

    elif timeline == 'last-month':
        
        date_filter = (df_incident['resolved_at'] < month_first_day) & (df_incident['resolved_at'] >= last_month)
        df = create_trbs_df(date_filter)
        return df

    elif timeline == 'current-year':
        date_filter = (df_incident['resolved_at'] >= year_first_day) & (df_incident['resolved_at'] < '2200-01-01')
        df = create_trbs_df(date_filter)
        return df

    elif timeline == 'ytd':

        date_filter = (df_incident['resolved_at'] < '2200-01-01')
        df = create_trbs_df(date_filter)
        return df

In [27]:
# Initialize trbs DataFrame
df_by_size_cm = df_ticket_resolved_by_size('current-month')
df_by_size_lm = df_ticket_resolved_by_size('last-month')
df_by_size_cy = df_ticket_resolved_by_size('current-year')
df_by_size_ytd = df_ticket_resolved_by_size('ytd') 

# df_by_size_cm
# df_by_size_lm
# df_by_size_cy
# df_by_size_ytd

In [28]:
# Create DataTables
dt_by_size_cm = dash_table.DataTable(
    df_by_size_cm.to_dict('records'),
    columns = [{"name": i, "id": i} for i in df_by_size_cm.columns]
)

dt_by_size_lm = dash_table.DataTable(
    df_by_size_lm.to_dict('records'),
    columns = [{"name": i, "id": i} for i in df_by_size_lm.columns]
)

dt_by_size_cy = dash_table.DataTable(
    df_by_size_cy.to_dict('records'),
    columns = [{"name": i, "id": i} for i in df_by_size_cy.columns]
)

dt_by_size_ytd = dash_table.DataTable(
    df_by_size_ytd.to_dict('records'),
    columns = [{"name": i, "id": i} for i in df_by_size_ytd.columns]
)



In [29]:
row2_cm = dbc.Row([
    dbc.Col(dt_by_size_cm, width=4),
    dbc.Col(dcc.Graph(figure=fig_df_assessment_cm), width=4),
    dbc.Col([
        dbc.Row(dcc.Graph(figure=fig_pie)),
        dbc.Row(dcc.Graph(figure=fig_pie))
    ], width=4),
])

row2_lm = dbc.Row([
    dbc.Col(dt_by_size_lm, width=4),
    dbc.Col(dcc.Graph(figure=fig_df_assessment_lm), width=4),
    dbc.Col([
        dbc.Row(dcc.Graph(figure=fig_pie)),
        dbc.Row(dcc.Graph(figure=fig_pie))
    ], width=4),
])

row2_cy = dbc.Row([
    dbc.Col(dt_by_size_cy, width=4),
    dbc.Col(dcc.Graph(figure=fig_df_assessment_cy), width=4),
    dbc.Col([
        dbc.Row(dcc.Graph(figure=fig_pie)),
        dbc.Row(dcc.Graph(figure=fig_pie))
    ], width=4),
])

row2_ytd = dbc.Row([
    dbc.Col(dt_by_size_ytd, width=4),
    dbc.Col(dcc.Graph(figure=fig_df_assessment_ytd), width=4),
    dbc.Col([
        dbc.Row(dcc.Graph(figure=fig_pie)),
        dbc.Row(dcc.Graph(figure=fig_pie))
    ], width=4),
])

In [30]:
# Create server variable with Flask server object for use with gunicorn
server = app.server

# Construct App Layout
app.layout = html.Div(children=[
    html.H1('Analytics Platform Ticket Dashboard'),
    dcc.Tabs(id="tabs-content-input", value='current-month', children=[
        dcc.Tab(label='Current Month', value='current-month'),
        dcc.Tab(label='Last Month', value='last-month'),
        dcc.Tab(label='Current Year', value='current-year'),
        dcc.Tab(label='Year to Date', value='ytd')
    ]),
    html.Br(),
    html.Div(id='tabs-content-output'),

])

@app.callback(Output('tabs-content-output', 'children'),
              Input('tabs-content-input', 'value'))
def render_content(tab):
    if tab == 'current-month':
        return html.Div([
            dbc.Row(row1_cm, justify="around"),
            html.Br(),
            dbc.Row(row2_cm),
        ])
    elif tab == 'last-month':
        return html.Div([
            dbc.Row(row1_lm, justify="around"),
            html.Br(),
            dbc.Row(row2_lm),  
        ])
    elif tab == 'current-year':
        return html.Div([
            dbc.Row(row1_cy, justify="around"),
            html.Br(),
            dbc.Row(row2_cy),  
        ])
    elif tab == 'ytd':
        return html.Div([
            dbc.Row(row1_ytd, justify="around"),
            html.Br(),
            dbc.Row(row2_ytd),  
        ])

app.run_server()

Dash app running on http://127.0.0.1:8050/
